# 程序功能增强

`BrainState` 框架基于[`pygraph` 语法](./pygraph-zh.ipynb)提供了强大的程序功能增强机制，允许用户在基本计算模型的基础上添加额外的功能，如自动微分、批处理、多设备并行等。本教程将详细介绍如何使用这些功能增强特性来优化和扩展你的模型。在阅读本教程之前，建议先阅读[`pygraph` 语法](./pygraph-zh.ipynb)教程。

## 1. 自动微分

自动微分是深度学习中最基础和最重要的功能之一。`BrainState`基于JAX的自动微分系统，提供了[简单直观的API](../apis/augment.rst)来计算梯度。


### 1.1 自动微分语法


`brainstate`提供的自动微分接口需要用户提供所需要求导的``State``集合。基本语法如下：

```python
gradients = brainstate.augment.grad(loss_fn, states)
```

其中，`loss_fn`是损失函数，`states`是需要求导的参数集合。`grad`函数返回一个新的函数，这个函数的输入和`loss_fn`相同，但是返回值是`states`中每个参数的梯度。`grad`针对标量损失函数进行求导，针对别的形式的损失函数，可以将其替换为其它形式的求导函数。目前我们支持的自动微分接口有：

- `brainstate.augment.grad`：标量损失函数的自动微分，使用反向模式自动微分（reverse-mode automatic differentiation）
- `brainstate.augment.vector_grad`：向量函数的自动微分，使用反向模式自动微分
- `brainstate.augment.jacrev`：标量函数的雅可比矩阵，使用反向模式自动微分
- `brainstate.augment.jacfwd`：标量函数的雅可比矩阵，使用前向模式自动微分（forward-mode automatic differentiation）
- `brainstate.augment.jacobian`：标量函数的雅可比矩阵，与`brainstate.augment.jacrev`等价
- `brainstate.augment.hessian`：标量函数的海森矩阵，使用反向模式自动微分
- 更多详细信息请参考[API文档](../apis/augment.rst)


`brainstate`提供的自动微分接口支持返回损失函数值(``return_value=True``)和支持返回辅助数据(auxiliary data, `has_aux=True`)。

当``return_value=True``时，返回值是一个元组，第一个元素是梯度，第二个元素是损失函数值。


```python
gradients, loss = brainstate.augment.grad(loss_fn, states)
```


当``has_aux=True``时，返回值是一个元组，第一个元素是梯度，第二个元素是辅助数据。此时，``loss_fn``需要返回一个元组，第一个元素是损失函数值，第二个元素是辅助数据。

```python
def loss_fn(*args):
    ...
    return loss, aux

gradients, aux = brainstate.augment.grad(loss_fn, states, has_aux=True)
```

当``return_value=True``和``has_aux=True``同时为True时，返回值是一个元组，第一个元素是梯度，第二个元素是损失函数值，第三个元素是辅助数据。

```python
def loss_fn(*args):
    ...
    return loss, aux

gradients, loss, aux = brainstate.augment.grad(loss_fn, states, return_value=True, has_aux=True)
```


### 1.2 基础梯度计算

`brainstate`提供的`grad`和`vector_grad`等函数支持一阶梯度计算。下面是一个简单的例子：


In [1]:
import jax.numpy as jnp

import brainunit as u
import brainstate

In [2]:
# 创建一个简单的线性层模型
model = brainstate.nn.Linear(2, 3)

# 准备输入数据
x = jnp.ones((1, 2))
y = jnp.ones((1, 3))


# 定义损失函数
def loss_fn(x, y):
    return jnp.mean((y - model(x)) ** 2)


# 获取模型参数
weights = model.states()

# 计算梯度
grads = brainstate.augment.grad(loss_fn, weights)(x, y)

# 打印梯度信息
print("Gradients:", grads)


Gradients: {('weight',): {'bias': Array([-0.66015077,  0.02036158, -1.7672682 ], dtype=float32), 'weight': Array([[-0.66015077,  0.02036158, -1.7672682 ],
       [-0.66015077,  0.02036158, -1.7672682 ]], dtype=float32)}}


### 1.3 高阶梯度计算

`BrainState`支持计算高阶导数，这在某些优化任务中非常有用：

In [3]:
# 计算二阶导数
hessian = brainstate.augment.hessian(loss_fn, weights)(x, y)

# 计算雅可比矩阵
jacobian = brainstate.augment.jacobian(loss_fn, weights)(x, y)

### 1.4 梯度变换和链式法则

你可以组合多个梯度计算操作：

In [4]:
# 组合多个梯度操作
def composite_grad(fn, params):
    grad_fn = brainstate.augment.grad(fn, params)
    return lambda *args: brainstate.augment.grad(grad_fn, params)(*args)

## 2. 批处理增强

批处理(Batching)是深度学习中提高计算效率的关键技术。它通过同时处理多个样本来提高硬件利用率，减少计算开销。``brainstate``支持对[``pygraph``模型](./pygraph-zh.ipynb)进行批处理，用户可以通过简单的API来实现批处理。

`brainstate.augment.vmap` 相比于 `jax.vmap`，添加了两个新的参数``in_states`` 和 ``out_states``，用于指定用到的模型中哪些``State``需要进行批处理。``in_states`` 和 ``out_states`` 是一个字典，键是批处理的维度，值是需要进行批处理的``State``（可以是任意的``State``组合的PyTree）。

与`jax.vmap`相同，`brainstate.augment.vmap`函数接收`in_axes`和`out_axes`参数，用于指定非`State`的参数的哪些维度是批处理的维度。`in_axes`和`out_axes`的用法与`jax.vmap`相同。

以下是一些简单的例子：

In [5]:
batch_size = 4

# 创建批处理数据和模型
x_batch = jnp.ones((batch_size, 3)) * u.mA
model = brainstate.nn.LIF(3)
model.init_state(batch_size)


# 对每一个批次计算损失
@brainstate.augment.vmap(
    in_states=model.states()  # 所有State都进行批处理
)
def eval(x):
    with brainstate.environ.context(dt=0.1 * u.ms):
        return model(x)

eval(x_batch)

Array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]], dtype=float32)


``brainstate.augment.vmap``会自动识别没有正确设置批处理维度的``State``，并给出警告。

In [6]:
class Foo(brainstate.nn.Module):
    def __init__(self):
        super().__init__()
        self.a = brainstate.ParamState(jnp.arange(4))
        self.b = brainstate.ShortTermState(jnp.arange(4))

    def __call__(self):
        self.b.value = self.a.value * self.b.value


foo = Foo()
r = brainstate.augment.vmap(foo, in_states=foo.a)()

BatchAxisError: The value of State ShortTermState(
  value=Traced<ShapedArray(int32[4])>with<BatchTrace> with
    val = Array([[0, 0, 0, 0],
         [0, 1, 2, 3],
         [0, 2, 4, 6],
         [0, 3, 6, 9]], dtype=int32)
    batch_dim = 0
) is batched, but it is not in the out_states.

在上面的例子中，我们定义了一个简单的模型`Foo`，只批处理了`a`，而没有批处理`b`。`brainstate`会自动检测到这个问题，并给出报错。正确的做法是将`b`也批处理，比如将其设置为`out_states`.


In [7]:
foo = Foo()
r = brainstate.augment.vmap(foo, in_states=foo.a, out_states=foo.b)()

foo.b.value

Array([[0, 0, 0, 0],
       [0, 1, 2, 3],
       [0, 2, 4, 6],
       [0, 3, 6, 9]], dtype=int32)

## 3. 多设备并行计算


不仅仅是批处理，`brainstate`还支持多设备并行计算。我们可以通过`brainstate.augment.pmap`函数将一个模型转换为支持多设备并行计算的模型。

`brainstate.augment.pmap`增强函数的用法基本上跟`brainstate.augment.vmap`函数是一样的，只是`pmap`函数会将模型转换为支持多设备并行计算的模型，而`vmap`函数只是但设备上不同线程上的并行计算。

## 4. 组合使用功能增强

在实际应用中，我们常常需要组合使用多种功能增强。``brainstate``中各种功能增强函数和编译函数之间是可以互相组合使用的。下面是一个简单的例子：


In [8]:
batch_size = 5
xs = brainstate.random.rand(batch_size, 3)
ys = brainstate.random.rand(batch_size, 4)

net = brainstate.nn.Linear(3, 4)

@brainstate.augment.vmap
def batch_run(x):
    return net(x)


def loss_fn(x, y):
    return jnp.mean((y - batch_run(x)) ** 2)


weights = net.states(brainstate.ParamState)
opt = brainstate.optim.Adam(1e-3)
opt.register_trainable_weights(weights)


@brainstate.compile.jit
def batch_train(xs, ys):
    grads, l = brainstate.augment.grad(loss_fn, weights, return_value=True)(xs, ys)
    opt.update(grads)
    return l


l = batch_train(xs, ys)
l

Array(0.9318466, dtype=float32)

## 5. 性能优化建议

使用功能增强时，需要注意以下几点以获得最佳性能：

1. **合理的批大小**：根据设备内存和计算能力选择适当的批大小
2. **梯度累积**：当批大小受限时，考虑使用梯度累积
3. **缓存编译**：重复使用已编译的函数以减少编译开销
4. **内存管理**：使用`jax.device_get()`及时释放设备内存


## 6. 调试技巧

在使用功能增强时，调试是一个重要话题。`brainstate`完全支持jax中提供的调试工具，如`jax.debug`中的`print`函数。下面是一个简单的例子：

```python
# 使用jax.debug.print进行调试
@bst.compile.jit
def batch_train(xs, ys):
    grads, l = bst.augment.grad(loss_fn, weights, return_value=True)(xs, ys)
    jax.debug.print("Gradients: {g}", g=grads)
    opt.update(grads)
    return l

```

详细用法用户可以参考[JAX调试文档](https://jax.readthedocs.io/en/latest/debugging/index.html)。


## 总结

基于`pygraph`的程序功能增强是`BrainState`框架的核心特性之一，通过合理使用这些功能，可以显著提升模型的推理、训练效率和性能。本教程涵盖了主要的功能增强特性及其使用方法，为进一步探索和应用这些特性提供了基础。